# NoSQL databases

**Deadline:** 24/05/2022 om 23u59

In deze notebook staan een aantal oefeningen over verschillende NoSQL databases.

## Wide-column database - Cassandra

Het eerste deel van de oefeningen gaat over Wide-column NoSQL Databases.
De structuur hiervan lijkt het sterkst op SQL databases.
Een veelgebruikte implementatie hiervan is [Cassandra](https://cassandra.apache.org/_/index.html).
De Cassandra database zou reeds geinstalleerd moeten zijn en kan gestart worden in een terminal door het volgende commando uit te voeren:

    cassandra

Controleer in de cell hieronder de status van cassandra, hoe kan je aan de output zien of cassandra goed opgestart is?

In [174]:
!nodetool status

nodetool: Failed to connect to '127.0.0.1:7199' - URISyntaxException: 'Malformed IPv6 address at index 7: rmi://[127.0.0.1]:7199'.


**Controleer de status van cassandra:**

Normaal gezien zou dit kunnen met "nodetool status" maar dit werkt niet bij mij. Cassandra is wel gewoon correct opgestart vermits ik kan verbinden en queries uitvoeren via het python package dat we hierna moeten installeren. PS: Ik heb ook heel het 1e deel kunnen maken met Cassandra dus ik ga er vanuit dat het werkt :)

**Antwoord:**

Behalve rechtstreekse queries te schrijven voor het Cassandra DBMS, is er ook een python package dat kan connecteren met een Cassandra DBMS. 
De API van deze package kan je [hier](https://docs.datastax.com/en/developer/python-driver/3.25/getting_started/) vinden.
Importeer deze package en connecteer met de lokale cassandra applicatie in de cell hieronder

In [2]:
!pip install cassandra-driver

In [3]:
from cassandra.cluster import Cluster

Met behulp van het Cassandra Database Management Systeem gaan we een database maken voor een Movie Streaming website. 
Maak in de onderstaande cell eerst deze keyspace aan en zorg ervoor dat de "Simpel Strategy" gekozen word en zet de replication factor op 1 (1 omdat we lokaal werken hier en niet op een cluster).
Voorzie ook een query om de beschikbare keyspaces op te vragen om te verifieren dat het correct is toegevoegd.
Zorg er daarna voor dat de keyspace ook eerst verwijderd wordt (enkel als ze reeds bestaat) en dat je op het einde deze keyspace als default selecteert.

In [16]:
# Verbinding maken met de Cassandra-cluster
cluster = Cluster(['localhost'])
session = cluster.connect()

# Keyspace naam
keyspace_name = "movie_streaming"

# Controleren op beschikbare keyspaces
list_keyspaces_query = "SELECT keyspace_name FROM system_schema.keyspaces"
keyspaces = session.execute(list_keyspaces_query)

# Keyspace verwijderen als deze al bestaat
if keyspace_name in [row.keyspace_name for row in keyspaces]:
    drop_keyspace_query = f"DROP KEYSPACE {keyspace_name}"
    session.execute(drop_keyspace_query)

# Keyspace aanmaken met SimpleStrategy en replication factor 1
create_keyspace_query = f"CREATE KEYSPACE {keyspace_name} WITH REPLICATION = {{ 'class' : 'SimpleStrategy', 'replication_factor' : 1 }}"
session.execute(create_keyspace_query)

# Keyspace als standaard selecteren
use_keyspace_query = f"USE {keyspace_name}"
session.execute(use_keyspace_query)

# Controleren op beschikbare keyspaces
list_keyspaces_query = "SELECT keyspace_name FROM system_schema.keyspaces"
keyspaces = session.execute(list_keyspaces_query)

# Printen van beschikbare keyspaces
print("Beschikbare keyspaces:")
for row in keyspaces:
    print(row.keyspace_name)

Beschikbare keyspaces:
system_auth
system_schema
system_distributed
system
system_traces
movie_streaming


Voor deze website willen we 3 tabellen aanmaken, namelijk voor de users, de beschikbare films en om gegeven scores bij te houden.
Al deze tabellen moeten gegroepeerd worden in een bepaalde keyspace. 
De naam hiervan mag je zelf kiezen.
Zorg voor minstens de volgende kolommen in de tabellen:
* Users
 * Id
 * Naam
 * Email
 * Land
 * Geboortedatum
* Films
 * Id
 * Naam
 * Genre
 * Duur (in minuten)
 * Beschrijving
* Ratings
 * UserID
 * MovieID
 * Score
 * Timestamp
 
Bij Cassandra zijn er verschillende manieren om de tabellen te sorteren, namelijk partitions, primary keys en  clustering columns.
Wat is het verschil tussen partitions en clustering columns? Om welke reden is het belangrijk om hiervoor een goede keuze te maken?

**Antwoord:**
Sorteren op partitie houdt in dat gegevens binnen partities worden georganiseerd op basis van een specifieke kolom of kolommen. Hierdoor worden gegevens met vergelijkbare waarden in de partitiekolom(s) samen opgeslagen, wat de queryprestaties kan verbeteren door de hoeveelheid gegevens die voor een specifieke query moet worden geopend te verminderen.

Sorteren op clusteringkolommen daarentegen houdt in dat de gegevens binnen elke partitie fysiek worden geordend op basis van één of meer clusteringkolommen. Deze rangschikking kan de queryprestaties verder verbeteren door gegevenslokalisatie te optimaliseren en de hoeveelheid gegevens die voor een specifieke query vanaf de schijf moet worden gelezen te minimaliseren.

Het is belangrijk om de juiste techniek te kiezen omdat dit direct van invloed is op de efficiëntie en prestaties van gegevensopvraagoperaties. Sorteren op partitie is effectief wanneer er een duidelijke partitiekolom is die kan worden gebruikt om gegevens te scheiden, terwijl sorteren op clusteringkolommen aanvullende voordelen biedt door gegevens binnen partities te organiseren. De keuze hangt af van de specifieke vereisten van de uitgevoerde queries en de aard van de dataset.

Schrijf hieronder de nodige code om bovenstaande tabellen aan te maken, de correcte datatypes te kiezen en de keys in te stellen.
Controleer dit door na het aanmaken van de tabellen ook de nodige code te schrijven om alle aanwezige tabellen uit te printen.

Let op voor de volgende zaken bij het toekennen van primary keys:
* Heel veel queries worden uitgevoerd op de gebruikers van een bepaald land
* Ook zijn er veel queries op de Films tabel die filteren op genre

In [123]:
# Tabel Users aanmaken
session.execute("DROP TABLE IF EXISTS Users")

session.execute("""CREATE TABLE Users (
    Id UUID,
    Naam TEXT,
    Email TEXT,
    Land TEXT,
    Geboortedatum DATE,
    PRIMARY KEY (Id, Land)
)""")

# Tabel Films aanmaken
session.execute("DROP TABLE IF EXISTS Films")

session.execute("""CREATE TABLE Films (
    Id UUID,
    Naam TEXT,
    Genre TEXT,
    Duur INT,
    Beschrijving TEXT,
    PRIMARY KEY (Id, Duur)
) WITH CLUSTERING ORDER BY (Duur DESC)""")


# Tabel Ratings aanmaken
session.execute("DROP TABLE IF EXISTS Ratings")

session.execute("""CREATE TABLE Ratings (
    MovieID UUID,
    UserID UUID,
    Score DECIMAL,
    Timestamp TIMESTAMP,
    PRIMARY KEY (MovieID, UserID)
)""")

# Controleren op aanwezige tabellen
list_tables_query = "SELECT table_name FROM system_schema.tables WHERE keyspace_name = 'movie_streaming'"
tables = session.execute(list_tables_query)

# Printen van aanwezige tabellen
print("Aanwezige tabellen:")
for row in tables:
    print(row.table_name)

Aanwezige tabellen:
films
ratings
users


Schrijf hieronder de nodige queries om minstens 3 users, 3 films en 5 scores toe te voegen.
Vul hierbij nooit de geboortedatum van de users en de beschrijving van de films in.
Zorg hierbij voor voldoende variatie in gebruikerslanden/genres en scores

In [124]:
import uuid

# Users
uuid1 = uuid.uuid1()
uuid2 = uuid.uuid1()
uuid3 = uuid.uuid1()

session.execute("""
    INSERT INTO Users (Id, Naam, Email, Land)
    VALUES (%s, %s, %s, %s)""",
    (uuid1, "Quinten", "quinten@studentodisee.be", "België"))

session.execute("""
    INSERT INTO Users (Id, Naam, Email, Land)
    VALUES (%s, %s, %s, %s)""",
    (uuid2, "Iejan", "iejan@ordina.com", "België"))

session.execute("""
    INSERT INTO Users (Id, Naam, Email, Land)
    VALUES (%s, %s, %s, %s)""",
    (uuid3, "Hanz", "hanz@audi.com", "Duitsland"))

# Films
     
uuid4 = uuid.uuid1()
uuid5 = uuid.uuid1()
uuid6 = uuid.uuid1()

session.execute("""
    INSERT INTO Films (Id, Naam, Genre, Duur)
    VALUES (%s, %s, %s, %s)""",
    (uuid4, "Interstellar", "Sci-Fi", 170))

session.execute("""
    INSERT INTO Films (Id, Naam, Genre, Duur)
    VALUES (%s, %s, %s, %s)""",
    (uuid5, "Inception", "Mind-bending", 230))

session.execute("""
    INSERT INTO Films (Id, Naam, Genre, Duur)
    VALUES (%s, %s, %s, %s)""",
    (uuid6, "The Godfather", "Mafia", 140))

In [125]:
# Scores

from faker import Faker
fake = Faker()

session.execute("""
    INSERT INTO Ratings (MovieID, UserID, Score, Timestamp)
    VALUES (%s, %s, %s, %s)""",
    (uuid4, uuid1, 3.0, int(fake.date_time_between(start_date='-3y', end_date='now').timestamp())))

session.execute("""
    INSERT INTO Ratings (MovieID, UserID, Score, Timestamp)
    VALUES (%s, %s, %s, %s)""",
    (uuid4, uuid2, 10.0, int(fake.date_time_between(start_date='-3y', end_date='now').timestamp())))

session.execute("""
    INSERT INTO Ratings (MovieID, UserID, Score, Timestamp)
    VALUES (%s, %s, %s, %s)""",
    (uuid5, uuid2, 9.0, int(fake.date_time_between(start_date='-2y', end_date='now').timestamp())))

session.execute("""
    INSERT INTO Ratings (MovieID, UserID, Score, Timestamp)
    VALUES (%s, %s, %s, %s)""",
    (uuid6, uuid2, 6.0, int(fake.date_time_between(start_date='-7y', end_date='now').timestamp())))

session.execute("""
    INSERT INTO Ratings (MovieID, UserID, Score, Timestamp)
    VALUES (%s, %s, %s, %s)""",
    (uuid5, uuid3, 8.0, int(fake.date_time_between(start_date='-4y', end_date='now').timestamp())))

session.execute("""
    INSERT INTO Ratings (MovieID, UserID, Score, Timestamp)
    VALUES (%s, %s, %s, %s)""",
    (uuid4, uuid3, 6.0, int(fake.date_time_between(start_date='-9y', end_date='now').timestamp())))

Om te controleren of de data goed is toegevoegd, schrijf hieronder de nodige code om de data in de tabellen/column families uit te printen.

In [126]:
print('Users')

rows = session.execute('SELECT * FROM Users')
for row in rows:
    print(row)
    
    
print('Films')

rows = session.execute('SELECT * FROM Films')
for row in rows:
    print(row)
    
    
print('Ratings')

rows = session.execute('SELECT * FROM Ratings')
for row in rows:
    print(row)

Users
Row(id=UUID('1b78c346-f7da-11ed-8003-b1118fa20c9b'), land='België', email='iejan@ordina.com', geboortedatum=None, naam='Iejan')
Row(id=UUID('1b78bf2c-f7da-11ed-8003-b1118fa20c9b'), land='België', email='quinten@studentodisee.be', geboortedatum=None, naam='Quinten')
Row(id=UUID('1b78c5f8-f7da-11ed-8003-b1118fa20c9b'), land='Duitsland', email='hanz@audi.com', geboortedatum=None, naam='Hanz')
Films
Row(id=UUID('1b7ae57c-f7da-11ed-8003-b1118fa20c9b'), duur=170, beschrijving=None, genre='Sci-Fi', naam='Interstellar')
Row(id=UUID('1b7ae86a-f7da-11ed-8003-b1118fa20c9b'), duur=230, beschrijving=None, genre='Mind-bending', naam='Inception')
Row(id=UUID('1b7ae9d2-f7da-11ed-8003-b1118fa20c9b'), duur=140, beschrijving=None, genre='Mafia', naam='The Godfather')
Ratings
Row(movieid=UUID('1b7ae57c-f7da-11ed-8003-b1118fa20c9b'), userid=UUID('1b78bf2c-f7da-11ed-8003-b1118fa20c9b'), score=Decimal('3.0'), timestamp=datetime.datetime(1970, 1, 19, 14, 35, 39, 28000))
Row(movieid=UUID('1b7ae57c-f7da-1

Nu dat er een aantal rijen testdata in de database aanwezig is, kunnen een aantal veelgebruikte queries geschreven worden.
Gebruik hiervoor prepared statemens voor de volgende zaken op te vragen:
* Selecteer alle gebruikers binnen een bepaald land (het land is een parameter van het prepared statement)
* Bereken de gemiddelde score van de films (Het resultaat moet het film id en gemiddelde score bevatten)
* Geef een overzicht van alle beschikbare films, gesorteerd volgens de duurtijd met de langste films eerst.
* Kies een bepaalde film, bereken de beste score gegeven aan deze gebruiker door een gebruiker uit een bepaald land. **Kan dit gedaan worden in 1 query? Indien niet, waarom niet? Hoe heb je dit aangepakt?**

**Antwoord:** Nee het kan niet in een query omdat er geen joins zijn. Ik heb eerst de film id opgehaald voor film naam, en dan een lijst van user ids waar land overeenkomt met het land dat we willen. Dan heb ik de maximale score genomen waar movieid overeenkomt en userid in de lijst is van userids uit het land dat we willen.

In [147]:
# code voor de queries
from cassandra.query import PreparedStatement

# 1
query = "SELECT * FROM Users WHERE Land = ? ALLOW FILTERING"
prepared_statement = session.prepare(query)

country_parameter = 'België'
rows = session.execute(prepared_statement, [country_parameter])

print(f"Users met Land = {country_parameter}:")
for row in rows:
    print(row)
    
# 2 voor deze vraag moest ik score int naar score decimal veranderen en ipv 3 bv 3.0 ingeven anders werden
# de gemiddeldes als int gegeven en dus altijd afgerond wat niet optimaal is
print("\nGemiddelde score per movie uuid:")

query = "SELECT MovieID, AVG(Score) as average_rating FROM Ratings GROUP BY MovieID ALLOW FILTERING"
prepared_statement = session.prepare(query)

rows = session.execute(prepared_statement)

for row in rows:
    print(row)
    
# 3
print("\nAlle films gesorteert op langste films eerst:")
query = "SELECT * FROM Films"
result = session.execute(query)

films = list(result)
sorted_films = sorted(films, key=lambda film: film.duur, reverse=True)

for film in sorted_films:
    print(film)

# 4
country = "België"
movie = "The Godfather"

select_users_by_country = session.prepare("SELECT * FROM Users WHERE Land = ? ALLOW FILTERING")
select_highest_rating = session.prepare("SELECT MAX(Score) FROM Ratings WHERE MovieID = ? AND UserID IN ? ALLOW FILTERING")
select_movie_id = session.prepare("SELECT Id FROM Films WHERE Naam = ? ALLOW FILTERING")

users_result_set = session.execute(select_users_by_country, [country])

listofuserids = list()
for row in users_result_set:
    listofuserids.append(row.id)

movieid = 0
movie_id_result_set = session.execute(select_movie_id, [movie])
for row in movie_id_result_set:
    movieid = row.id

highest_rating_result_set = session.execute(select_highest_rating, [movieid, listofuserids])
highest_rating = highest_rating_result_set.one().system_max_score

if(highest_rating == None):
    print(f"\nGeen ratings gevonden voor film: {movie} uit gebruikers uit land: {country}")
else:
    print(f"\nHoogste rating voor film: {movie} uit gebruikers uit land: {country} is: {highest_rating}")

Users met Land = België:
Row(id=UUID('1b78c346-f7da-11ed-8003-b1118fa20c9b'), land='België', email='iejan@ordina.com', geboortedatum=None, naam='Iejan')
Row(id=UUID('1b78bf2c-f7da-11ed-8003-b1118fa20c9b'), land='België', email='quinten@studentodisee.be', geboortedatum=None, naam='Quinten')

Gemiddelde score per movie uuid:
Row(movieid=UUID('1b7ae57c-f7da-11ed-8003-b1118fa20c9b'), average_rating=Decimal('6.3'))
Row(movieid=UUID('1b7ae86a-f7da-11ed-8003-b1118fa20c9b'), average_rating=Decimal('8.5'))
Row(movieid=UUID('1b7ae9d2-f7da-11ed-8003-b1118fa20c9b'), average_rating=Decimal('6.0'))

Alle films gesorteert op langste films eerst:
Row(id=UUID('1b7ae86a-f7da-11ed-8003-b1118fa20c9b'), duur=230, beschrijving=None, genre='Mind-bending', naam='Inception')
Row(id=UUID('1b7ae57c-f7da-11ed-8003-b1118fa20c9b'), duur=170, beschrijving=None, genre='Sci-Fi', naam='Interstellar')
Row(id=UUID('1b7ae9d2-f7da-11ed-8003-b1118fa20c9b'), duur=140, beschrijving=None, genre='Mafia', naam='The Godfather')



## Key - value database - Redis

Een tweede belangrijk type NoSQL database is een key-value database.
Hierbij wordt de data opgeslagen als key-value pairs wat dus sterk lijkt op het concept van dictionaries.
Een database systeem van dit type dat in de praktijk vaak gebruikt wordt is Redis.
Dit is reeds geinstalleerd op je virtuele machine.
Naast het aanbieden van een shell om manuele queries uit te voeren is er ook een sterk gelijkaardige python package, namelijk [redis-py](https://github.com/redis/redis-py).
Op een aantal uitzondering na zijn de python functies analoog met de standaard Redis functies die [hier](https://redis.io/commands) opgelijst zijn.

De oefening om het werking met een key-value NoSQL database is dat we een voorbeeld gaan maken van een webshop.
Kies zelf minstens drie producten die je wilt verkopen in je winkel.
Deze producten hebben de volgende gegevens:
* Naam
* Type
* Kleur
* Gewicht
* Prijs
* Voorraad
* Aantal verkocht

Voeg nu deze producten toe aan een redis database. 
Let hierbij op voor de volgende zaken:
* Beperk het aantal keer dat er commando's verstuurd worden om de communicatietijd te minimaliseren
* Gebruik een tag in de key om de producten te groeperen en gebruik een willekeurig ID als id van de producten. 

In [170]:
# maak een redis database aan en voeg een aantal producten toe
import redis

r = redis.Redis()
r.mset({"{1}Id": str(uuid.uuid1()), "{1}Naam": "Asus Rog 3070ti", "{1}Type": "GPU", "{1}Kleur": "Zwart/RGB verlichting", "{1}Gewicht": "735g", "{1}Prijs": "1750.00", "{1}Voorraad": 75, "{1}Aantal verkocht": 6})
r.mset({"{2}Id": str(uuid.uuid1()), "{2}Naam": "Intel i9 12700k", "{2}Type": "CPU", "{2}Kleur": "Grijs", "{2}Gewicht": "15g", "{2}Prijs": "659.99", "{2}Voorraad": 60, "{2}Aantal verkocht": 12})
r.mset({"{3}Id": str(uuid.uuid1()), "{3}Naam": "Corsair 32gb DDR4", "{3}Type": "RAM", "{3}Kleur": "Wit/RGB verlichting", "{3}Gewicht": "350g", "{3}Prijs": "75.50", "{3}Voorraad": 40, "{3}Aantal verkocht": 60})

True

Schrijf nu een functie dat toelaat om een gegeven aantal van 1 type product te kopen.
Deze functie voert de volgende zaken uit:
* Controleer of er voldoende voorraad is
* Indien ja, verlaag de voorraad en tel het aantal verkochte product op
* Zorg ervoor dat er gecontroleerd wordt dat de voorraad niet gewijzigd is tussen de stappen (transacties/vermijden race-condities)
* Return of de aankoop gelukt is of niet.

In [171]:
# schrijf functie om een aantal items van 1 product te kopen
def ItemBestellen(Id):
    Voorraad = int(r.get(f'{Id}Voorraad'))
    AantalVerkocht = int(r.get(f'{Id}Aantal verkocht'))
    
    print(f"Resterende voorraad voor item {Id}: {Voorraad}")
    
    if(Voorraad > 0) :
        r.set(f'{Id}Voorraad', Voorraad - 1)
        r.set(f'{Id}Aantal verkocht', AantalVerkocht + 1)
        print('Bestelling geplaatst (voorraad met 1 gezakt, verkocht met 1 verhoogd)')
    else:
        print('Bestelling mislukt')

Koop nu een aantal willekeurige producten en controleer of de voorraad en aantal verkochtte producten na het aankopen van de producten correct is.
Plaats de code om dit uit te voeren hieronder:

In [172]:
# koop een aantal producten van verscheidene producten en controleer het aantal
ItemBestellen("{1}")
ItemBestellen("{2}")
ItemBestellen("{3}")

Resterende voorraad voor item {1}: 75
Bestelling geplaatst (voorraad met 1 gezakt, verkocht met 1 verhoogd)
Resterende voorraad voor item {2}: 60
Bestelling geplaatst (voorraad met 1 gezakt, verkocht met 1 verhoogd)
Resterende voorraad voor item {3}: 40
Bestelling geplaatst (voorraad met 1 gezakt, verkocht met 1 verhoogd)


Door een probleem met de leverancier is er 1 product niet meer leverbaar.
Het is nu nodig om deze key te verwijderen uit de database.
Doe dit nu maar zorg ervoor dat de key maar na 10 seconden verwijder wordt.

In [173]:
# verwijder 1 product, zorg voor een delay van 10 seconden
import time

time.sleep(10)

keys = r.keys("{3}*")
for key in keys:
    r.delete(key)

## Document databases - MongoDb

Een derde belangrijk type van NoSQL databases zijn gebaseerd op het concept van documenten.
Dit type is een subset van key-value databases wat meer optimalisaties toelaten voor het queryen op values.
MongoDB is een populaire implementatie van dit type databases.
Een package om binnen python te werken met MongoDb is pymongo.
Meer informatie kan je [hier](https://pymongo.readthedocs.io/en/stable/) vinden.
Met behulp van onderstaand commando kan de mongodb gestart worden.

    mongod

Dit commando start de MongoDb Server. Zolang deze applicatie draait kan je met MongoDb connecteren via een shell (niet geinstalleerd) of een api zoals pymongo.

In deze repository staat ook een json file met een reeks documenten die ingeladen kunnen worden in een database.
Open de json eerst in een teksteditor en bekijk de gegevens die erin staan.
Laad nu de documenten in de cell hieronder in door de volgende stappen uit te voeren:
* Connecteer met mongodb
* Laad de json file in
* Maak een database aan in mongodb met naam "database"
* Maak een collectie aan met de naam restaurants
* Laad de data in de json file in in de net aangemaakte collectie
* Print het aantal ingeladen documenten uit (het aantal documenten in de collectie)

In [1]:
!pip install pymongo

Defaulting to user installation because normal site-packages is not writeable


In [3]:
import json
import pymongo

Client = pymongo.MongoClient()
DataBase = Client.Restaurant

Restaurant = DataBase.Restaurant
Restaurant.drop()

Rijen = []
with open('restaurants.json') as f:
    Rijen = f.readlines()
    for rij in Rijen:
        JSON_Rij_Ingelezen = json.loads(rij)
        Restaurant.insert_one(JSON_Rij_Ingelezen)
        
print(len(Rijen))

3772


Op basis van deze dataset kunnen een hele reeks queries uitgevoerd worden.
Schrijf hieronder de nodige queries om de volgende zaken op te zoeken:
* Toon van de eerste 5 documenten in de collectie de naam, soort keuken (cuisine) en wijk (borough). Zorg er ook voor dat 
* Toon een lijst met enkel de namen, straat en huisnummer van de bakkerijen in de collectie. Zorg ervoor dat enkel het 14e tot 27e document getoond wordt. 
* Toon de namen van alle Chinese restauranten met een score tussen 50 en 80.
* Toon de top 5 restauranten met een Amerikaanse keuken op basis van de gemiddelde score. Let erop dat de hoogste score bovenaan staat en zorg ervoor dat enkel de naam van het restaurant, zijn restaurant_id (en niet het _id veld) en gemiddelde score getoond wordt.
* Geef een lijst met alle keukens in de dataset.
* Tel het aantal restaurants per soort keuken. Sorteer dit in aflopende volgorde en toon enkel de naam van de keuken en het aantal.
* Hoeveel restauranten hebben in hun naam de tekst "chez" staan (niet noodzakelijk vooraan en niet hoofdlettergevoelig?
* Geef een overzicht van enkel de namen van alle restaurants die minstens 7 "A" grades hebben gehad.

Zorg bij het opstellen van deze queries er ook voor dat de resultaten uitgeprint worden (indien het er maximaal 20 zijn).
De dataset is in het Engels opgesteld.
Indien een bepaalde vertaling niet duidelijk is kan je steeds de json in een tekst editor openen 

In [6]:
# query 1 er staat "zorg er ook voor dat" maar er staat niks achter?
resultaten = Restaurant.find({}, {"_id": 0, "name": 1, "cuisine": 1, "borough": 1}).limit(5)
for resultaat in resultaten:
    print(resultaat)

{'borough': 'Bronx', 'cuisine': 'Bakery', 'name': 'Morris Park Bake Shop'}
{'borough': 'Brooklyn', 'cuisine': 'Hamburgers', 'name': "Wendy'S"}
{'borough': 'Manhattan', 'cuisine': 'Irish', 'name': 'Dj Reynolds Pub And Restaurant'}
{'borough': 'Brooklyn', 'cuisine': 'American ', 'name': 'Riviera Caterer'}
{'borough': 'Queens', 'cuisine': 'Jewish/Kosher', 'name': 'Tov Kosher Kitchen'}


In [9]:
# query 2
resultaten = Restaurant.find({ "cuisine": "Bakery" }, { "_id": 0, "name": 1, "address.street": 1, "address.building": 1 }).skip(13).limit(14)
for resultaat in resultaten:
    print(resultaat)

{'address': {'building': '4202', 'street': 'White Plains Road'}, 'name': 'E & L Bakery & Coffee Shop'}
{'address': {'building': '15622', 'street': 'Cross Bay Boulevard'}, 'name': 'Sugar Bun Bakery'}
{'address': {'building': '385', 'street': 'Broome Street'}, 'name': 'Caffe Roma'}
{'address': {'building': '4007', 'street': 'National Street'}, 'name': 'La Antioquena Bakery Ii'}
{'address': {'building': '29', 'street': 'Hugh Grant Circle'}, 'name': "Zaro'S Bread Basket"}
{'address': {'building': '80-02', 'street': 'Surrey Place'}, 'name': 'Bagels & Cream Deli'}
{'address': {'building': '2923', 'street': 'Avenue R'}, 'name': "Michael'S Pastry Shop"}
{'address': {'building': '194', 'street': 'Canal Street'}, 'name': 'Tai Pan Bakery'}
{'address': {'building': '10830', 'street': 'Ascan Avenue'}, 'name': 'Bonelle Pastry Shop'}
{'address': {'building': '289', 'street': 'Manhattan Avenue'}, 'name': 'Fortunato Bros Cafe & Bakery'}
{'address': {'building': '3331', 'street': 'Francis Lewis Boulevar

In [10]:
# query 3
resultaten = Restaurant.find({ "cuisine": "Chinese", "grades.score": { "$gt": 50, "$lt": 80 } }, { "_id": 0, "name": 1})
for resultaat in resultaten:
    print(resultaat)

{'name': 'May May Kitchen'}
{'name': "Lee'S Villa Chinese Restaurant"}
{'name': 'Hop Won Express'}
{'name': 'Dynasty'}
{'name': 'Red Star Chinese Restaurant'}


In [23]:
# query 4
resultaten = Restaurant.aggregate([ { "$match": { "cuisine": "American " }}, { "$project": { "_id": 0, "name": 1, "restaurant_id": 1, "avg_score": { "$avg": "$grades.score" } } }, { "$sort": { "avg_score": -1 }}, { "$limit": 5 } ])
for resultaat in resultaten:
    print(resultaat)

{'name': 'West 79Th Street Boat Basin Cafe', 'restaurant_id': '40756344', 'avg_score': 36.0}
{'name': "Murals On 54/Randolphs'S", 'restaurant_id': '40372466', 'avg_score': 33.666666666666664}
{'name': 'Live Bait Bar & Restaurant', 'restaurant_id': '40387237', 'avg_score': 32.6}
{'name': "Billy'S Sport Bar Restaurant & Lounge", 'restaurant_id': '40624470', 'avg_score': 30.6}
{'name': 'Tobacco Road', 'restaurant_id': '40785901', 'avg_score': 29.833333333333332}


In [24]:
# query 5
resultaten = Restaurant.find({}, {"_id":0, "cuisine":1}).distinct("cuisine")
print(resultaten)

['Afghan', 'African', 'American ', 'Armenian', 'Asian', 'Bagels/Pretzels', 'Bakery', 'Bangladeshi', 'Barbecue', 'Bottled beverages, including water, sodas, juices, etc.', 'Brazilian', 'CafÃ©/Coffee/Tea', 'Café/Coffee/Tea', 'Caribbean', 'Chicken', 'Chinese', 'Chinese/Cuban', 'Chinese/Japanese', 'Continental', 'Czech', 'Delicatessen', 'Donuts', 'Eastern European', 'Egyptian', 'English', 'Ethiopian', 'Filipino', 'French', 'German', 'Greek', 'Hamburgers', 'Hotdogs', 'Ice Cream, Gelato, Yogurt, Ices', 'Indian', 'Indonesian', 'Irish', 'Italian', 'Japanese', 'Jewish/Kosher', 'Juice, Smoothies, Fruit Salads', 'Korean', 'Latin (Cuban, Dominican, Puerto Rican, South & Central American)', 'Mediterranean', 'Mexican', 'Middle Eastern', 'Moroccan', 'Not Listed/Not Applicable', 'Other', 'Pakistani', 'Pancakes/Waffles', 'Peruvian', 'Pizza', 'Pizza/Italian', 'Polish', 'Portuguese', 'Russian', 'Salads', 'Sandwiches', 'Sandwiches/Salads/Mixed Buffet', 'Seafood', 'Soul Food', 'Soups & Sandwiches', 'Spanis

In [26]:
# query 6 wat bedoelt u met aflopende volgorde? aflopend op naam of count? ik ga er vanuit aflopend op count, is iets logischer dan op naam
resultaten = Restaurant.aggregate([ { "$group": { "_id": "$cuisine", "count": { "$sum": 1 } } }, { "$sort": { "count": -1 } } ])
for resultaat in resultaten:
    print(resultaat)

{'_id': 'American ', 'count': 1255}
{'_id': 'Italian', 'count': 325}
{'_id': 'Pizza', 'count': 270}
{'_id': 'Café/Coffee/Tea', 'count': 180}
{'_id': 'Hamburgers', 'count': 159}
{'_id': 'Bakery', 'count': 127}
{'_id': 'Latin (Cuban, Dominican, Puerto Rican, South & Central American)', 'count': 115}
{'_id': 'Chinese', 'count': 115}
{'_id': 'Pizza/Italian', 'count': 106}
{'_id': 'Japanese', 'count': 80}
{'_id': 'Irish', 'count': 79}
{'_id': 'Delicatessen', 'count': 78}
{'_id': 'Caribbean', 'count': 75}
{'_id': 'Mexican', 'count': 73}
{'_id': 'French', 'count': 72}
{'_id': 'Jewish/Kosher', 'count': 60}
{'_id': 'Donuts', 'count': 43}
{'_id': 'Indian', 'count': 43}
{'_id': 'Spanish', 'count': 42}
{'_id': 'Seafood', 'count': 35}
{'_id': 'Bagels/Pretzels', 'count': 34}
{'_id': 'Chicken', 'count': 32}
{'_id': 'Greek', 'count': 25}
{'_id': 'Ice Cream, Gelato, Yogurt, Ices', 'count': 25}
{'_id': 'Steak', 'count': 21}
{'_id': 'Korean', 'count': 21}
{'_id': 'Sandwiches', 'count': 19}
{'_id': 'Sandw

In [35]:
# query 7
resultaat = Restaurant.count_documents({"name": {"$regex": "(?i).*chez.*", "$options": "i"}})
print(f"Er zijn {resultaat} restaurants met chez in hun naam, niet case sensitive en kan overal staan in de naam.")

Er zijn 3 restaurants met chez in hun naam, niet case sensitive en kan overal staan in de naam.


In [40]:
resultaten = Restaurant.aggregate([ {"$unwind": "$grades"}, {"$match": {"grades.grade": { "$regex": "A"}}}, {"$group": { "_id": "$_id", "name": {"$first": "$name"}, "count": {"$sum": 1}, }}, {"$match": {"count": {"$gte": 7}}}, {"$project": { "_id": 0, "name": 1, "count": 1 }} ])
for resultaat in resultaten:
    print(resultaat)

{'name': "Dunkin' Donuts/Baskin Robins", 'count': 7}
{'name': 'Boston Market', 'count': 7}
{'name': "Cafe Amore'S Restaurant", 'count': 7}
{'name': 'French Roast', 'count': 7}
{'name': "Munchy'S", 'count': 7}
{'name': 'Cafe Metro', 'count': 7}
{'name': 'Ritz Diner', 'count': 7}
{'name': 'Rincon Salvadoreno Restaurant', 'count': 7}
{'name': 'Fifth Ave Cafe /Diner', 'count': 7}
{'name': 'Cafe Bonjour', 'count': 7}
{'name': "Andrew'S Diner", 'count': 7}
{'name': 'Kelley & Ping', 'count': 7}
{'name': 'Almar', 'count': 7}
{'name': 'El Ranchito De Daisy', 'count': 7}
{'name': 'Bread Market Cafe', 'count': 7}
{'name': "Dunkin' Donuts, Baskin Robbins", 'count': 7}
{'name': 'El Olivo', 'count': 7}
{'name': 'Igloo Ice Cream Cafe', 'count': 7}
{'name': 'Blue Mountain Restaurant & Jerk Center', 'count': 7}
{'name': 'Coffee Shop', 'count': 7}
{'name': 'Broadway Pizza & Pasta', 'count': 7}
{'name': "Mcdonald'S", 'count': 7}
{'name': 'Taj Donut Shop', 'count': 7}
{'name': 'Ho Mei Restaurant', 'count'